# Ayudantía 11: 
## Iterables y Funcionales
### Autores: [Patricio Hinostroza](https://github.com/dvckhv) - [Lucas VSJ](https://github.com/lucasvsj)

## Contenidos:

- Iterables
- Generadores
- lambda functions
- map - filter - reduce


# Iterables

## Iterar

Antes de comenzar la ayudantía, definamos el concepto de **Iterar**.

Segun [Wikipedia](https://es.wikipedia.org/wiki/Iteraci%C3%B3n), podemos definir **iteración** como: 

"...repetir varias veces un proceso con la intención de alcanzar una meta deseada, objetivo o resultado"

Ahora llevándolo a código, **iterar** sería poder acceder a los elementos de una estructura de datos aplicando esencialmente algún algoritmo. 

## Ahora veámoslo en nuestra propia estructura de datos
Utilicemos una estructura de datos que ya conocemos, una `Lista Ligada`. Como ya vimos en ayudantias pasadas, podemos almacenar datos de manera secuencial en ella, pero la forma de acceder a ellos es un poco ineficiente, ya que debemos acceder a cada `Nodo` para poder avanzar hacia el siguiente.

In [1]:
class Nodo:
    
    def __init__(self, valor, siguiente):
        # Cada nodo contiene un valor...
        self.valor = valor
        # ... y referencia al siguiente Nodo
        self.siguiente = siguiente
    
    def __repr__(self):
        return f"{self.valor}"

In [2]:
# Avanzar de un nodo a otro
my_ll = Nodo(1, Nodo(2, Nodo(3, Nodo(4, Nodo(5, None)))))
valor_0 = my_ll.valor
valor_1 = my_ll.siguiente.valor
valor_2 = my_ll.siguiente.siguiente.valor
print(f"{valor_0}, {valor_1}, {valor_2}")

1, 2, 3


Podemos notar que este método se vuelve tedioso rápidamente a medida que queremos ir accediendo a datos que se encuentran más lejanos a la cabeza de nuestra `Lista Ligada`. Para solucionar esto, la mejor forma que tenemos es *iterar* sobre la estructura, pero actualmente no podemos hacerlo.

In [3]:
for item in my_ll:
    print(item)

TypeError: 'Nodo' object is not iterable

Para poder solucionar esto, debemos "transformar" nuestra estructura en un `Iterable`, para así, poder **iterar** sobre esta.

## Iterables

Un `Iterable` es una estructura de datos sobre la cual podemos **iterar**, es decir, acceder a sus elementos de manera secuencial.


### Build-ins iterables
- Listas
- Sets
- Diccionarios
- Deque
- Strings

Un iterable tiene definido el metodo `__iter__()` para poder ser iterable. Veamos el `Iterable` más famoso de `Python`, la `lista`. 

In [4]:
my_lista = [1,2,3]
l = [print(char) if char == '__iter__' else None for char in my_lista.__dir__()]
print(len(l))

__iter__
46


Como podemos ver a continuación, podemos **iterar** sobre `my_lista`

In [5]:
for elem in my_lista:
    print(elem)

1
2
3


Esto lo logramos gracias al `Iterador` que está dentro de nustro `Iterable` (`my_lista`).

In [6]:
print(my_lista.__iter__())

## Iteradores

- Se define como un objeto que itera sobre un iterable.

- El método `__iter__()` de un `Iterable` **debe** retonar su `Iterador`  y este debe tener definido el método `__next__()`.

- Cuando el `Iterable` no tiene más elementos por recorrer con el método `__next__()` el objeto debe levantar una excepción del tipo `StopIteration`

In [7]:
L = list(range(9))
iterador = iter(L)
print(type(iterador))
for _ in range(10):
    print(next(iterador),end = ", ")

<class 'list_iterator'>
0, 1, 2, 3, 4, 5, 6, 7, 8, 

StopIteration: 

### Los iteradores se "consumen"

Cada vez que iteramos sobre una estructura de datos debemos generar un nuevo `Iterador`, debido a que estos no se reinician una vez terminan de recorrer el objeto:

In [8]:
print(next(iterador),end = ", ")


StopIteration: 

## Podemos crear una estructura iterable

Como mencionamos, para crear nuestro propio Iterable, necesitamos 2 conceptos claves:

* Almacenar el objeto sobre el cual queremos iterar (en nuestro caso seria la `Lista Ligada`).
* Implementar un `Iterador` dentro del método `__iter__`, el cual se encargara de iterar sobre nuestro `Iterable`.

Comenzamos creando nuestro propio `Iterador`, el cual presenta 3 conceptos claves:

* Almacenar el `Iterable` dentro de suyo.
* Implementar el método `__iter__`, el cual solo retorna a sí mismo
* Implementar el método `__next__`, el cual se encarga de entregar el siguiente elemento del Iterable y de levantar una `Exception` si es que se llega al final de nuestro Iterable (no quedan más elementos por los cuales iterar).


In [9]:
class ListaLigada:
    
    def __init__(self, objeto):
        self.cabeza = objeto
    
    def __iter__(self):
        return IteradorLL(self.cabeza)
    
class IteradorLL:
    
    def __init__(self, iterable):
        self.iterable = iterable
    
    def __iter__(self): # Siempre un iterador se retorna a si mismo aquí
        return self
    
    def __next__(self, echo=False): 
        if self.iterable is None:
            # Levantamos una excepción del tipo StopIteration
            # con el mensaje "Llegamos al final".
            raise StopIteration("Llegamos al final")
        else:
            if echo:
                print("Vamos a ir a buscar el siguiente Nodo de la LL")           
            valor = self.iterable
            self.iterable = valor.siguiente
            return valor

### Iteremos sobre nuestra Lista Ligada 

In [10]:
iterable = ListaLigada(my_ll)
for i in iterable:
    print(i)

print("EMPEZAMOS DE NUEVO")
for i in iterable:
    print(i)

1
2
3
4
5
EMPEZAMOS DENUEVO
1
2
3
4
5


## Ya podemos iterar sobre nuestra propia estructura de datos!


## Generadores

Los generadores son un caso especial de los iteradores. Los generadores nos permiten iterar sobre secuencias de datos sin la necesidad de almacenarlos en alguna estructura especial, evitando el uso innecesario de memoria

In [11]:
gen = (i for i in range(10))
print(type(gen))
print(next(gen))
print(next(gen))

<class 'generator'>
0
1


## Uso de memoria

In [12]:
from sys import getsizeof
for size in range(4):
    print(f"Comparando Tamaño {10**size}")
    g = (i**2 for i in range(10**size))
    print("Tamaño Generador en bytes:" + str(getsizeof(g)))
    l = [i**2 for i in range(10**size)]
    print("Tamaño lista en bytes:" + str(getsizeof(l)))
    print("-"*10)

Comparando Tamaño 1
Tamaño Generador en bytes:112
Tamaño lista en bytes:88
----------
Comparando Tamaño 10
Tamaño Generador en bytes:112
Tamaño lista en bytes:184
----------
Comparando Tamaño 100
Tamaño Generador en bytes:112
Tamaño lista en bytes:904
----------
Comparando Tamaño 1000
Tamaño Generador en bytes:112
Tamaño lista en bytes:9016
----------


## Funciones Generadoras

Las funciones en Python también tienen la posibilidad de funcionar como generadores, con la sentencia `yield`. 

El *statement* `yield` se encarga de **retornar** el valor, pero además guarda un estado, por lo que el siguiente llamado a la función partirá donde quedó en el llamado anterior. 

### Creemos un generador para nuestra lista ligada

In [13]:
from copy import deepcopy
def recorrido_LL(LL):
    nodo = LL.cabeza
    while nodo != None:
        yield nodo.valor
        nodo = nodo.siguiente

gen = recorrido_LL(ListaLigada(my_ll)) #no comienza iterando!
print(gen)
print("Primera iteración")
for valor in gen:
    print(valor,end = " ")
print("\nSegunda iteración")

for valor in gen:
    print(valor,end = " ")
print("Los generadores se Consumen")

gen = recorrido_LL(ListaLigada(my_ll)) 
print(list(gen))
print(list(gen))

<generator object recorrido_LL at 0x000001837E2C92E0>
Primera iteración
1 2 3 4 5 
Segunda iteración
Los generadores se Consumen
[1, 2, 3, 4, 5]
[]


## Enviar valores a Funciones Generadoras

Al igual que es posible sacar elementos de mi generador, tambien puedo enviarle información para cada iteración gracias al *statement* `yield` y el método `.send(data)` propio del generador.

In [14]:
class ListaLigada_V2:
    def __init__(self,nodo):
        self.cabeza = nodo
        self.cola = nodo
    # ahora ponemos nuestro metodo generador como __iter__
    def __iter__(self):
        nodo = self.cabeza
        while nodo != None:
            yield nodo
            nodo = nodo.siguiente

    def añadir_nodo(self):
        print("Iniciamos generador")
        while True:
            nodo = yield
            self.cola.siguiente = nodo
            self.cola = self.cola.siguiente
            print(f"se añadió el nodo de valor {nodo.valor} a la cola")

my_LL2 = ListaLigada_V2(Nodo(1,None))
print("se crea el generador")
manejo_nodos = my_LL2.añadir_nodo()
print("aun no se inicia")
next(manejo_nodos)

for i in range(3,10):
    manejo_nodos.send(Nodo(i,None))

print("ahora iteramos en nuestra Lista Ligada")
for nodo in my_LL2:
    print(nodo.valor,end = ", ")
            


se crea el generador
aun no se inicia
Iniciamos generador
se añadió el nodo de valor 3 a la cola
se añadió el nodo de valor 4 a la cola
se añadió el nodo de valor 5 a la cola
se añadió el nodo de valor 6 a la cola
se añadió el nodo de valor 7 a la cola
se añadió el nodo de valor 8 a la cola
se añadió el nodo de valor 9 a la cola
ahora iteramos en nuestra Lista Ligada
1, 3, 4, 5, 6, 7, 8, 9, 

## Built-ins 

Python tiene muchas funciones *built-ins utiles*, veremos los siguientes:
- `__getitem__`: permite acceder a elementos de algun objeto mediante el método de indexación, las `Listas` y `Strings` tienen este método definido.

- `__len__`: permite retornar el largo de un objeto, viene definida en todos las estructuras *built-in* de python
- `__reversed__`: permite retornar una copia de un objeto iterable en orden inverso, es necesario que `__getitem__` y `__len__` se encuentren definidos por defecto. Se puede hacer *override*.

- `enumerate`: retorna una tupla, en la que el primer elemento es el índice que va enumerando, y el segundo es el ítem original que está siendo numerado.

- `zip`: toma dos o más iterables y junta su contenido en tuplas por índice.

### Implementemos algunos *Built-ins* en nuestra Lista Ligada

In [15]:
class ListaLigada_V3:
    def __init__(self,nodo):
        self.cabeza = nodo
        self.cola = nodo
    def __iter__(self):
        nodo = self.cabeza
        while nodo != None:
            yield nodo
            nodo = nodo.siguiente

    def añadir_nodo(self):
        while True:
            nodo = yield
            self.cola.siguiente = nodo
            self.cola = self.cola.siguiente
            print(f"se añadió el nodo de valor {nodo.valor} a la cola")
            
    #nos aprovechamos de que las listas ya tienen esto implementado
    # podemos hacerlo "a mano" pero es mas largo

    def __len__(self):
        gen = iter(self)
        return len(list(gen))

    def __getitem__(self,i):
        gen = iter(self)
        return list(gen)[i]

 # podemos definir conversión de tipos tambien
    def __list__(self):
        return list(iter(self)) 

    def __int__(self):
        suma = 0
        for nodo in self:
            suma+=nodo.valor
        return suma

LL_3 = ListaLigada_V3(Nodo(1,None))
manejo_nodos = LL_3.añadir_nodo()
next(manejo_nodos)
list(manejo_nodos.send(Nodo(i,None)) for i in range(3,10)) # esto tambien es un 

print(len(LL_3))
print("el cuarto elemento de mi Lista Ligada es",LL_3[4])
print(list(LL_3))
print(list(reversed(LL_3)))
print(int(LL_3))

se añadió el nodo de valor 3 a la cola
se añadió el nodo de valor 4 a la cola
se añadió el nodo de valor 5 a la cola
se añadió el nodo de valor 6 a la cola
se añadió el nodo de valor 7 a la cola
se añadió el nodo de valor 8 a la cola
se añadió el nodo de valor 9 a la cola
8
el cuarto elemento de mi Lista Ligada es 6
[1, 3, 4, 5, 6, 7, 8, 9]
[9, 8, 7, 6, 5, 4, 3, 1]
43


### Uso de enumerate

In [16]:
for i,nodo in enumerate(LL_3):
    print(f"el valor del nodo de la posición {i} es {nodo.valor}")

el valor del nodo de la posición 0 es 1
el valor del nodo de la posición 1 es 3
el valor del nodo de la posición 2 es 4
el valor del nodo de la posición 3 es 5
el valor del nodo de la posición 4 es 6
el valor del nodo de la posición 5 es 7
el valor del nodo de la posición 6 es 8
el valor del nodo de la posición 7 es 9


### Uso de zip

Para instruir correctamente el uso de zip utilizaremos otro ejemplo, ya que no es muy pertinente el uso de nodos.

In [17]:
vocales = ["a","e","i","o","u"]
numeros = range(1,10)
colores = ["azul","verde","amarillo","morado","rojo","negro","blanco"]
union = zip(vocales,numeros,colores)
# Se truncan los valores dado el objeto mas corto a unir, en este caso las vocales
print(list(union))

[('a', 1, 'azul'), ('e', 2, 'verde'), ('i', 3, 'amarillo'), ('o', 4, 'morado'), ('u', 5, 'rojo')]


# Programación Funcional

## Lambda Functions

Para poder hablar de `lambda functions`, primero tenemos que entender que `Python` tiene **funciones de primera clase** (first-class functions), es decir, que las funciones son tratadas como cualquier otra variable (**objeto**). Esto nos permite manejarlas de la misma forma que cualquier otro objeto de `Python` (como las listas por ejemplo), otorgándonos la posibilidad de:

* Asignarlas a una variable y usar la variable igual que una funcion.
* Pasarlas como párametros de otras funciones.


In [18]:
# Creamos funcion a aplicar sobre el iterable
def calcular_total(iterable):
    return sum(iterable)

In [19]:
# Definimos la funcion que aplica la funcion anterior
# sobre el iterable
def monto_boleta(montos, calculadora):
    resultado = calculadora(montos)
    print(f'El total de la boleta fue {resultado}')
    return resultado

In [20]:
# Vemos los resultados
precios = [1200, 3500, 2980, 5990, 7650]
monto_boleta(precios, calcular_total)
monto_boleta(my_lista, calcular_total)

El total de la boleta fue 21320
El total de la boleta fue 6


6

## Map

Map es una función *built-in* que **retorna un generador** creado a partir de la aplicacion de una **función** sobre cada elemento de uno o más **iterables**. Para esto, recibe como **parámetros** una **funcion** y uno a mas **iterables**.

In [21]:
from copy import deepcopy

# (Nos aseguramos de no cambiar la LL por
# cada ejecucion de esta celda)
iterable = ListaLigada(deepcopy(my_ll))

# Puede recibir tanto una funcion definida
def suma_de_a_pares(item_lista_1, item_lista_2):
    return item_lista_1.valor + item_lista_2.valor

def_func_result = map(suma_de_a_pares, iterable, iterable)


# Puede recibir una lambda function
lambda_func_result = map(lambda char: pow(char.valor, char.valor), iterable)

# Vemos los resultados
print(
    f'Generator with defined function: {def_func_result}',
    '\n', 
    f'Generator with lambda function: {lambda_func_result}',
    '\n', 
    sep='',
)

print("@lucasvsj's Tip: Pythonic way to see results once")

print(
    f'Generator with defined function: {[ char for char in def_func_result]}',
    '\n', 
    f'Generator with lambda function: {[ char for char in lambda_func_result]}',
    sep=''
)


Generator with defined function: <map object at 0x000001837C7F9190>
Generator with lambda function: <map object at 0x000001837C7061F0>

@lucasvsj's Tip: Pythonic way to see results once
Generator with defined function: [2, 4, 6, 8, 10]
Generator with lambda function: [1, 4, 27, 256, 3125]


## Filter

Esta funcion nos permite "filtrar" elementos de un iterable. Esto lo logra aplicando una función *booleana* sobre un **iterable**.

In [22]:
from copy import deepcopy

# (Nos aseguramos de no cambiar la LL por
# cada ejecucion de esta celda)
datos_1 = Nodo('_nombre', Nodo('Pedro', Nodo('_apellido', Nodo('Perez', Nodo('_score', Nodo(45, None))))))
iterable_1 = ListaLigada(deepcopy(datos_1))

# Puede recibir tanto una funcion definida
def datos_personales(item_lista):
    if isinstance(item_lista.valor, str):
        return item_lista.valor[0] != '_'
    return False

def_func_result = filter(datos_personales, iterable_1)


# Puede recibir una lambda function
lambda_func_result = filter(lambda char: isinstance(char.valor, int), iterable_1)

print(
    f'Generator with defined function: {def_func_result}',
    '\n', 
    f'Generator with lambda function: {lambda_func_result}',
    '\n', 
    sep='',
)

print(
    f'Generator with defined function: {[ char for char in def_func_result]}',
    '\n', 
    f'Generator with lambda function: {[ char for char in lambda_func_result]}',
    sep=''
)


Generator with defined function: <filter object at 0x000001837E2D9F40>
Generator with lambda function: <filter object at 0x000001837C701D00>

Generator with defined function: [Pedro, Perez]
Generator with lambda function: [45]


## Reduce

`reduce` es una función de la libreria *built-in* `functools`. Esta se caracteriza por aplicar una función de manera iterativa sobre el resultado del output de la función aplicada anteriormente

In [23]:
from functools import reduce

from copy import deepcopy

# (Nos aseguramos de no cambiar la LL por
# cada ejecucion de esta celda)
iterable_1 = ListaLigada(deepcopy(my_ll))

# Puede recibir tanto una funcion definida
def llevar_cuenta(valor_acum, valor_nuevo):
    if isinstance(valor_acum, Nodo):
        return valor_acum.valor + valor_nuevo.valor
    return valor_acum + valor_nuevo.valor

def_func_result = reduce(llevar_cuenta, iterable_1)

datos_1 = Nodo('Mi ', Nodo('nombre ', Nodo('es ', Nodo('Pedro ', Nodo('Perez ', None)))))
iterable_1 = ListaLigada(deepcopy(datos_1))

# Puede recibir una lambda function
lambda_func_result = reduce(lambda x, y: x.valor + y.valor if isinstance(x, Nodo) else x + y.valor, iterable_1)

print(
    f'Generator with defined function: {def_func_result}',
    '\n', 
    f'Generator with lambda function: {lambda_func_result}',
    '\n', 
    sep='',
)


Generator with defined function: 15
Generator with lambda function: Mi nombre es Pedro Perez 

